# Create Mongo Database Connection

In [ ]:
from pymongo import MongoClient
class DBConnector():
    def __init__(self, ip, db_name, usr,  psw):
        self.db_name = db_name
        self.usr = usr
        self.psw = psw
        self.uri = 'mongodb://{0}:{1}@{2}/{3}'.format(usr, psw, ip, db_name)
        print(self.uri)
    def mongoClient(self, collection):
        self.mongo_client = MongoClient(self.uri)
        db = self.mongo_client[self.db_name]
        db_client = db[collection]
        return db_client
    def close_connection(self):
        self.mongo_client.close()

In [ ]:
db_connector = DBConnector('10.6.13.139', 'twitter_merged', 'ruser1', 'PASSWORD')
collection = db_connector.mongoClient('politicians')

# Retrieve the list of politicians from the SMM API 

In [ ]:
from smm_wrapper import SMMPoliticians

# Create an instance to the smm wrapper
smm = SMMPoliticians()

# Request the politicians from the API
politicians = smm.dv.get_politicians()

politicians.head()

# Retrieve tweets of one politician
#### e.g. `politician_id` == 1 corresponds to Johann David Wadephul

In [ ]:
from datetime import datetime
import pandas as pd

try:
    # pandas >= 0.25
    from pandas.io.json._normalize import nested_to_record 
except:
    from pandas.io.json.normalize import nested_to_record   

def get_tweets_of_twitter_ids(twitter_ids, _from='2017-7-5', _to='2018-12-31'):
    query = collection.find({
        'user.id': {'$in': twitter_ids},
        'createdAt': {
            '$gte': datetime.strptime(_from, '%Y-%m-%d'), 
            '$lt': datetime.strptime(_to, '%Y-%m-%d'), 
        }
    })
    return pd.DataFrame(nested_to_record(x) for x in query)
    

In [ ]:
twitter_ids=politicians.loc[1, 'tw_ids']
get_tweets_of_twitter_ids(twitter_ids, '2017-7-5', '2018-12-31')

# Retrieve tweets of the first 100 politicians

In [ ]:
# Create a list of all twitter ids. Note that one politician could have
# different twitter ids
from IPython.display import clear_output

politician_dfs = []
for _, politician in politicians.iloc[0:100].iterrows():
    clear_output()
    print(f"Downloading tweets of {politician['firstname']} {politician['name']}")
    
    if politician['tw_ids'] != None:
        politician_dfs.append(get_tweets_of_twitter_ids(politician['tw_ids']))

In [ ]:
tweets = pd.concat(politician_dfs)
tweets.head()